This notebook allows you to select a single CCD from a Com Cam image, and then find GAIA sources based on mouse click. 

It runs using data at the summit so you either need to be there, or else be on summit VPN. Be sure to respect Rubin embargo restrictions. 

C. Stubbs Nov 2024

In [ ]:

from lsst.summit.utils.utils import checkStackSetup
checkStackSetup()

In [ ]:
import pylab as plt
from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay
from lsst.ip.isr.isrTask import IsrTask
from lsst.cp.pipe.cpCombine import CalibCombineTask
import traceback

In [ ]:
#repo_path = "/sdf/data/rubin/repo/main"
repo_path = "/repo/LSSTComCam"
butler = Butler(repo_path)
# collections = 'LSSTComCam/raw/all,LSSTComCam/calib'.split(",")

# snag ISR'd iamges
# on summit make this quickLook rather than nightlyValidation
collections = 'LSSTComCam/defaults,LSSTComCam/quickLook'.split(",")
butler = Butler(repo_path,collections=collections)
registry = butler.registry

Layout of sensors in ComCam:

S20     S21      S22

S10     S11      S12

S00     S01      S02

In [ ]:
# detector ID 4 as integer is center, or can use "R22_S00" nomenclature as string . S11 is center
# exp=butler.get("postISRCCD", exposure=2024102600100,detector="R22_S00")
#exp=butler.get("postISRCCD", exposure=2024102600100,detector="R22_S00")

getday=20241027
getseq=69

dataid={"seq_num":getseq,"day_obs":getday}

# exp=butler.get("postISRCCD", dataId=dataid,detector="R22_S11") # sensor 11 should be the one in the center of ComCam raft. 

exp=butler.get("calexp", dataId=dataid,detector="R22_S00") # this fetches an astrometrically registered image. 

# exp=butler.get("raw", dataId=dataid,detector="R22_S11") # this gets non-flat-fielded image

In [ ]:
# Import necessary Rubin Observatory packages to make an image with RA and DEC overlaid
from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay
from astropy.wcs import WCS
import astropy.units as u
import matplotlib.pyplot as plt
from lsst.geom import Point2D 
import numpy as np

# Assuming `exp` is already defined as an exposure
image_data = exp.image.array
skyWcs = exp.getWcs()  # Ensure this is a SkyWcs object

# Create an Astropy WCS object from the Rubin SkyWcs
wcs_header = skyWcs.getFitsMetadata()
astropy_wcs = WCS(wcs_header)

# Find the center pixel coordinates
center_y, center_x = np.array(image_data.shape) // 2
center_pixel = Point2D(center_x, center_y)

# Get the RA and Dec at the center in decimal degrees using SkyWcs
sky_coord = skyWcs.pixelToSky(center_pixel)
center_ra = sky_coord.getRa().asDegrees()
center_dec = sky_coord.getDec().asDegrees()
print(f"Center RA: {center_ra:.6f} degrees, Center Dec: {center_dec:.6f} degrees")

# Display the image with RA/Dec overlay and labels
fig, ax = plt.subplots(subplot_kw={'projection': astropy_wcs}, figsize=(10, 10))
im = ax.imshow(image_data, origin='lower', cmap='gray', vmin=np.percentile(image_data, 5), vmax=np.percentile(image_data, 95))
ax.set_xlabel('RA')
ax.set_ylabel('Dec')

# Overlay RA/Dec grid lines with finer labels
ax.coords.grid(color='red', ls='dotted')

# Configure RA and Dec axes for finer labeling
ra_axis = ax.coords[0]  # RA axis
dec_axis = ax.coords[1]  # Dec axis

# Set specific ticks with finer intervals
ra_axis.set_ticks(spacing=1.0 * u.arcmin)   # Set RA ticks every 1 arcminute
dec_axis.set_ticks(spacing=1.0 * u.arcmin)  # Set Dec ticks every 1 arcminute

# Enable tick labels and set labels for RA and Dec
ra_axis.set_ticklabel_visible(True)
ra_axis.set_axislabel("Right Ascension")
dec_axis.set_ticklabel_visible(True)
dec_axis.set_axislabel("Declination")

# Add a colorbar for reference
plt.colorbar(im, ax=ax, orientation='vertical', fraction=0.03, pad=0.04, label="Intensity")

# Text element for displaying RA and Dec
ra_dec_text = ax.text(0.05, 0.95, '', transform=ax.transAxes, color='yellow', fontsize=12, verticalalignment='top')

# Event handler to update the text with RA and Dec
def on_mouse_move(event):
    if event.inaxes == ax:
        x, y = event.xdata, event.ydata
        if x is not None and y is not None:
            # Convert pixel coordinates to SkyWcs and then RA/Dec
            pixel = Point2D(x, y)
            try:
                sky_coord = skyWcs.pixelToSky(pixel)
                ra = sky_coord.getRa().asDegrees()
                dec = sky_coord.getDec().asDegrees()
                ra_dec_text.set_text(f"RA: {ra:.6f}°, Dec: {dec:.6f}°")
            except Exception:
                ra_dec_text.set_text("")

# Connect the event handler
fig.canvas.mpl_connect('motion_notify_event', on_mouse_move)

plt.show()


In [ ]:
# Activate interactive backend
plt.close('all')
%matplotlib widget

# Imports
import matplotlib.pyplot as plt
import numpy as np
from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import astropy.units as u
from lsst.geom import Point2D
from astroquery.gaia import Gaia
from ipywidgets import Output
from IPython.display import display

# Sample image data (replace `exp` with your Rubin exposure object)
image = exp.image.array
skyWcs = exp.getWcs()  # SkyWcs object from Rubin

# Set display stretch based on percentiles
lower_percentile = 5
upper_percentile = 95
min_value = np.percentile(image, lower_percentile)
max_value = np.percentile(image, upper_percentile)

# Create figure and axis for the image display
fig, ax = plt.subplots(figsize=(12,12))
im = ax.imshow(image, origin='lower', cmap='gray', vmin=min_value, vmax=max_value)
ax.set_title("Click on the image to select a point and query GAIA")

# Widgets for output display
output = Output()
pos = []  # Storage for clicked positions

# Text element to display RA/Dec dynamically
ra_dec_text = ax.text(0.05, 0.95, '', transform=ax.transAxes, color='yellow', fontsize=12, verticalalignment='top')

# Declare `result` as a global variable to make it accessible in other cells
result = None

# Event handler for mouse movement to update RA/Dec readout
def on_mouse_move(event):
    if event.inaxes == ax:
        x, y = event.xdata, event.ydata
        if x is not None and y is not None:
            # Convert pixel coordinates to RA and Dec
            pixel = Point2D(x, y)
            try:
                sky_coord = skyWcs.pixelToSky(pixel)
                ra = sky_coord.getRa().asDegrees()
                dec = sky_coord.getDec().asDegrees()
                ra_dec_text.set_text(f"RA: {ra:.6f}°, Dec: {dec:.6f}°")
            except Exception:
                ra_dec_text.set_text("")

# Click event handler to store click position and query GAIA
def on_mouse_click(event):
    global result  # Declare result as global so it can be accessed outside this function
    if event.inaxes == ax:
        x, y = event.xdata, event.ydata
        if x is not None and y is not None:
            # Convert pixel coordinates to RA and Dec
            pixel = Point2D(x, y)
            try:
                sky_coord = skyWcs.pixelToSky(pixel)
                ra = sky_coord.getRa().asDegrees()
                dec = sky_coord.getDec().asDegrees()
                
                # Store the clicked position for further analysis
                pos.clear()
                pos.append((ra, dec))
                
                with output:
                    output.clear_output()  # Clear output before displaying new results
                    print(f"Clicked at RA: {ra:.6f}, Dec: {dec:.6f}")
                
                # Query GAIA catalog within a 15 arcsecond radius
                coord = SkyCoord(ra, dec, unit="deg")
                result = Gaia.query_object_async(coordinate=coord, radius=15 * u.arcsec)

                # Display the query result
                with output:
                    if len(result) > 0:
                        print("GAIA sources within 15 arcsec:")
                        
                        # Extract magnitudes from the result
                        g_mag = result['phot_g_mean_mag']
                        bp_mag = result['phot_bp_mean_mag']
                        rp_mag = result['phot_rp_mean_mag']

                        # Loop over each row and print G, BP, and RP magnitudes on the same line
                        print("g_mag, bp_mag, rp_mag")
                        for g, bp, rp in zip(g_mag, bp_mag, rp_mag):
                            print(f"{g:.2f}, {bp:.2f}, {rp:.2f}")
                    else:
                        print("No GAIA sources found within 15 arcsec.")

            except Exception as e:
                with output:
                    output.clear_output()
                    print("Error:", e)


# Connect event handlers
fig.canvas.mpl_connect('motion_notify_event', on_mouse_move)
fig.canvas.mpl_connect('button_press_event', on_mouse_click)

# Display output and interactive widgets
display(output)
plt.show()


In [ ]:
# Activate interactive backend
plt.close('all')
%matplotlib widget

# Imports
import matplotlib.pyplot as plt
import numpy as np
from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, Column
from lsst.geom import Point2D
from astroquery.gaia import Gaia
from ipywidgets import Output
from IPython.display import display

# Sample image data (replace `exp` with your Rubin exposure object)
image = exp.image.array
skyWcs = exp.getWcs()  # SkyWcs object from Rubin

# Set display stretch based on percentiles
lower_percentile = 5
upper_percentile = 95
min_value = np.percentile(image, lower_percentile)
max_value = np.percentile(image, upper_percentile)

# Create figure and axis for the image display
fig, ax = plt.subplots(figsize=(12,12))
im = ax.imshow(image, origin='lower', cmap='gray', vmin=min_value, vmax=max_value)
ax.set_title("Click on the image to select a point and query GAIA")

# Widgets for output display
output = Output()
pos = []  # Storage for clicked positions

# Text element to display RA/Dec dynamically
ra_dec_text = ax.text(0.05, 0.95, '', transform=ax.transAxes, color='yellow', fontsize=12, verticalalignment='top')

# Declare `result` as a global variable to make it accessible in other cells
result = None

# Create an empty table to store clicked star information
star_table = Table(names=("Pixel X", "Pixel Y", "RA", "DEC"), dtype=('i4', 'i4', 'f8', 'f8'))

# Event handler for mouse movement to update RA/Dec readout
def on_mouse_move(event):
    if event.inaxes == ax:
        x, y = event.xdata, event.ydata
        if x is not None and y is not None:
            # Convert pixel coordinates to RA and Dec
            pixel = Point2D(x, y)
            try:
                sky_coord = skyWcs.pixelToSky(pixel)
                ra = sky_coord.getRa().asDegrees()
                dec = sky_coord.getDec().asDegrees()
                ra_dec_text.set_text(f"RA: {ra:.6f}°, Dec: {dec:.6f}°")
            except Exception:
                ra_dec_text.set_text("")

# Click event handler to store click position and query GAIA
def on_mouse_click(event):
    global result  # Declare result as global so it can be accessed outside this function
    if event.inaxes == ax:
        x, y = event.xdata, event.ydata
        if x is not None and y is not None:
            # Convert pixel coordinates to RA and Dec
            pixel = Point2D(x, y)
            try:
                sky_coord = skyWcs.pixelToSky(pixel)
                ra = sky_coord.getRa().asDegrees()
                dec = sky_coord.getDec().asDegrees()
                
                # Add new row to the star table
                star_table.add_row((int(x), int(y), ra, dec))
                
                # Display the updated table in the output widget
                with output:
                    output.clear_output()  # Clear output before displaying new results
                    print(f"Clicked at Pixel (x, y): ({x:.0f}, {y:.0f})")
                    print(f"Clicked at RA: {ra:.6f}°, Dec: {dec:.6f}°")
                    print("Updated Star Table:")
                    display(star_table)
                
                # Query GAIA catalog within a 15 arcsecond radius
                coord = SkyCoord(ra, dec, unit="deg")
                result = Gaia.query_object_async(coordinate=coord, radius=15 * u.arcsec)

                # Display the query result
                with output:
                    if len(result) > 0:
                        print("GAIA sources within 15 arcsec:")
                        
                        # Extract magnitudes from the result
                        g_mag = result['phot_g_mean_mag']
                        bp_mag = result['phot_bp_mean_mag']
                        rp_mag = result['phot_rp_mean_mag']

                        # Loop over each row and print G, BP, and RP magnitudes on the same line
                        print("g_mag, bp_mag, rp_mag")
                        for g, bp, rp in zip(g_mag, bp_mag, rp_mag):
                            print(f"{g:.2f}, {bp:.2f}, {rp:.2f}")
                    else:
                        print("No GAIA sources found within 15 arcsec.")

            except Exception as e:
                with output:
                    output.clear_output()
                    print("Error:", e)


# Connect event handlers
fig.canvas.mpl_connect('motion_notify_event', on_mouse_move)
fig.canvas.mpl_connect('button_press_event', on_mouse_click)

# Display output and interactive widgets
display(output)
plt.show()
